# Azure Update to Local File

The script loads the latest data in the Azure kuzu DB and writes it to a local file ("feather file"). There is no trasformation, filtering or cleaning of the data.£

In [25]:
import pyodbc
import pandas as pd
import os
from dotenv import load_dotenv
import pickle
import pyarrow

In [14]:
## Load .env file
load_dotenv('../config/.env')

True

## Load data from Azure DB

In [15]:
## Import credentials for kuzu Azure DB from .env file
SERVER_AZURE = os.environ.get('SERVER_AZURE', "default")  
DATABASE_AZURE = os.environ.get('DATABASE_AZURE', "default")
USERNAME_AZURE = os.environ.get('USERNAME_AZURE', "default")
PASSWORD_AZURE = os.environ.get('PASSWORD_AZURE', "default")
DRIVER_AZURE = os.environ.get('DRIVER_AZURE', "default")

In [16]:
cnxn = pyodbc.connect('DRIVER='+DRIVER_AZURE+';SERVER='+SERVER_AZURE+';DATABASE='+DATABASE_AZURE+';UID='+USERNAME_AZURE+';PWD='+ PASSWORD_AZURE)
cursor = cnxn.cursor()

In [17]:
#Sample select query
table_names = [x[2] for x in cursor.tables(tableType='TABLE')]
print(table_names)

['Antwort', 'Bahnhof', 'BahnhofUnique', 'BahnhofZuBahnhofUnique', 'Frage', 'Frageversion', 'FrageZuGruppe', 'Gruppe', 'Ortsgewicht', 'Ortskonzept', 'Region', 'Teilnehmer', 'Umfrage', 'WegPunkt', 'WegStrecke', 'trace_xe_action_map', 'trace_xe_event_map']


### Select relevant data to export

In [18]:
## Define relevant survey
UmfrageName = 'kuzu_zug'

In [19]:
q = f"""SELECT FrageCode FROM Frage WHERE UmfrageName LIKE '{UmfrageName}';""" # We want all FrageCodes here
cols =  pd.read_sql(q, con=cnxn)
col_list =  cols.FrageCode.values.tolist()

## Add manual cols of interest
col_list.insert(0, "file_name")
col_list.insert(0, "UmfrageName")
col_list.insert(0, "participant")
col_list.insert(0, "time")

In [20]:
cols = ', '.join(col_list)

In [21]:
## Bild SQL query
query = f"""SELECT {cols} FROM Teilnehmer WHERE UmfrageName LIKE '{UmfrageName}';"""

In [22]:
## Load data
kuzu_zug =  pd.read_sql(query , con=cnxn)

### Write dataframe to file

In [24]:
kuzu_zug.to_feather("../data/Data_Raw_Kuzu_Zug") # store data in feather file